# TPM Quality control

Input: 
1.A TPM file, with gene_ID as first column and each samples ID as a subsquent column (Similar to a bed file)
2.A gene count file that are to be made consistant with the TPM file

Output:
1. A set of three diagnostic plot from each of the outlier detection method
2. A TPM file with low expression gene, samples with missing data, and outlier removed

In [ ]:
[global]
## Inputs
parameter: TPM_file = path
parameter: gene_count_file = path
parameter: wd = path
## Enviorment
parameter: name = "demo"
parameter: container = ""
## Parameters
parameter: RLEFilterPercent0.05
parameter: DSFilterPercent = 0.05
parameter: topk_genes = 100
parameter: cluster_percent = 0.6
parameter: pvalues.cut = 0.05
parameter: cluster_level = 5
parameter: low_expr_TPM = 0.1
parameter: low_expr_TPM_percent = 0.2
parameter: TPM_pseudo_count = 0.01

In [1]:
[QC]
input: TPM_file
output: f'{wd}/{name}.qced.tpm'
R: expand = "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout',container = container 
    # Load packages
    library(RColorBrewer)
    library(ape)
    library(reshape2)
    library(tidyverse)
    ## Setting inputs
    tpmfile <- $[TPM_file]
    
    ## Setting parameters
    RLEFilterPercent <-$[RLEFilterPercent]
    DSFilterPercent <- $[DSFilterPercent]
    # Hcluster parameter
    pvalues.cut <- $[pvalues.cut]
    topk_genes <- $[topk_genes]
    cluster_percent <- $[cluster_percent]
    treesNum <- $[cluster_level]
    # 
      
    low_expr_TPM <- $[low_expr_TPM]
    low_expr_TPM_percent <- $[low_expr_TPM_percent]
    pseudo_count <- $[TPM_pseudo_count]
    print("Configuration file load successfully!")
    
    ## QC
    ### Data integrity check
    TPM_data <- read_delim(csvfile,header = TRUE,delim = "\t") # The input file should has sample ID as column name; the first column is gene ID 
    names(TPM_data)[1] <- "feature"
    if( sum(duplicated(TPM_data$feature)) > 0){
      print("Feature ID (e.g. gene names) should be in the first column. Please remove duplicated feature IDs, Exit!")
      quit(save = "no", status = 1, runLast = FALSE)
    }else{
      rownames(TPM_data) <- TPM_data$feature
      TPM_data = TPM_data[,-1]
      loaded_sample_count <- ncol(TPM_data)
    }
    #### make sure every expr column is in numeric type
    matrix_check <- map(TPM_data, is.numeric) %>% unlist
    if(sum(!matrix_check) > 0){
      print("The following column(s) in expression matrix is/are NOT in numeric type. Plese check, Proceed by excluding those samples")
      print(paste(names(matrix_check)[!matrix_check], collapse = "; "))
      TPM_data = TPM_data[,matrix_check]
      }
    print("Gene expression profiles loaded successfully!")
    print(paste(nrow(TPM_data), "genes and", ncol(TPM_data), "samples are loaded from", csvfile, sep = " "))
    
    #### Filter out low exp genes
    keep_genes_idx <- (rowMeans(TPM_data>low_expr_TPM)>low_expr_TPM_percent) 
    TPM_data = TPM_data[keep_genes_idx,]%>%as.matrix
    print(paste(sum(1 - keep_genes_idx), "genes are filtered, because >", low_expr_TPM_percent*100, "% samples have expression values <", low_expr_TPM))
    print(paste(sum(keep_genes_idx), "genes left."))
    logtpm = log10(TPM_data + pseudo_count)
    
    RLEFilterLength <- RLEFilterPercent*ncol(TPM_data)
    DSFilter <- DSFilterPercent*ncol(TPM_data)
    
    ## Outlier detection
    ### RLE
    rle=logtpm-apply(logtpm, 1, median) # change "/" to "-" so that we got log(fold-change) which centered on 0 on the RLE plot.
    iqr = apply(rle,2,IQR)
    rle=melt(cbind(ID=rownames(rle), rle), variable.name = "Sample",value.name ="TPM", id="ID")
    names(rle) <- c("feature","Sample","TPM")
    pdf(file = "$[_output[0]:n].RLEplot.pdf")
    boxplot(TPM ~ bymedian, data=rle, outline=F, ylim = c(-rle_IQR_range, rle_IQR_range), las=2, boxwex=1, col='gray', cex.axis=0.3, main="RLE plot before QC", xlab="", ylab="Residual expression levels", frame=F)
    dev.off()
    rle_IQR <- rle %>% group_by(Sample) %>% summarise(IQR = IQR(TPM))
    rle_IQR_range <- rle_IQR$IQR %>% range %>% abs() %>% max()
    rle_IQR_range <- 2*rle_IQR_range %>% ceiling()
    bymedian <- with(rle, reorder(Sample, TPM, IQR))  # sort by IQR
    par(mar=c(3,3,3,3))
    ExpPerSample <- nrow(TPM_data)
    RLEFilterList <- unique(bymedian[((length(bymedian)-ExpPerSample*RLEFilterLength)+1):length(bymedian)]) #filtered
    RLEFilterList <- as.character(RLEFilterList)
    print(paste0("The right most ", RLEFilterPercent*100, "% samples (N = ", length(RLEFilterList), ") are marked as candidate outliers in this step:") )
    RLEFilterList

    
    ### hcluster
    
    sampleDists <- 1 - cor(logtpm, method='spearman')
    hc <- hclust(as.dist(sampleDists), method = "complete")
    hcphy <- as.phylo(hc)
  
    pdf(file = "$[_output[0]:n].preQC_cluster.pdf")
    plot(hcphy, type = "unrooted", cex=.2, lab4ut='axial',underscore = T, main="Sample clustering before QC (Spearman - Cor.)")
    dev.off()
    ntop <- topk_genes
    Pvars <- apply(logtpm, 1, var)
    select <- order(Pvars, decreasing =TRUE)[seq_len(min(ntop, length(Pvars)))]
    MD_matrix <- logtpm[select, ]
    MahalanobisDistance = mahalanobis(t(MD_matrix), colMeans(t(MD_matrix)), cov(t(MD_matrix))) 
    # Note: t(MD_matrix)=sample_row*gene_column, Manhalanobis() returns one vector with length=row number
    pvalues = pchisq(MahalanobisDistance, df=nrow(MD_matrix), lower.tail=F)
    pvalues.adjust = p.adjust(pvalues, method ="bonferroni") # adjusted pvalues for each sample
    pvalues.low <- pvalues.adjust[pvalues.adjust<pvalues.cut]
    
    HCoutliers <- character()
    for(x in c(1:treesNum)){
      trees <- cutree(hc,k=x)
      idx <- c(1:x)#which tree is checking
      for(i in idx)
      {
        group <- hc$labels[which(trees == i)]
        if(sum(group %in% names(pvalues.low))/length(group) >= cluster_percent)
        {
          HCoutliers <- union(HCoutliers,group)
        }
      }
    }
    
    print(paste(length(HCoutliers), "samples are marked as candidate outlier(s) in this step.", sep = " "))
    if(length(HCoutliers)>0){
      print("Sample outliers are marked in red as follows:")
      print(HCoutliers)
      co1 = hc$labels%in%HCoutliers
      co1[which(co1 == "FALSE")]<-"gray0"
      co1[which(co1 == "TRUE")]<-"red"
      par(mar=c(3,3,3,3))
  
    pdf(file = "$[_output[0]:n].cluster.pdf")
    plot(hcphy, tip.col = co1, type = "unrooted", cex=.2, lab4ut='axial',underscore = T, main="Label Outliers in Red")
    Xcol = c("gray0", "red")
    Xtext = c("Normal Sample", "Outliers")
    legend('bottomleft',pch=21,Xtext, col='white',pt.bg=Xcol, cex=1)
    dev.off()
    }else{
      print("No outlier detected.")
    }
    
    ### D-s
    D = apply(1-sampleDists, 1, median)
    pdf(file = "$[_output[0]:n].D-stat_hist.pdf")
    hist(D, breaks=100, ylab="Number of samples", xlab="D-statistic", main="Histogram of Sample D-statistics before data QC")
    dev.off()
    
    DSFilter <- sort(D)[DSFilter]
    D<-as.data.frame(D)
    D<-data.frame(Sample = rownames(D),D = D$D)
    D_filterList = D%>%filter(D <= DSFilter)
    D_filterList <- D_filterList$Sample
    D_filterList<-as.character(D_filterList)
    print(paste0("The right most ", DSFilterPercent*100, "% samples (N=", length(D_filterList), ") are marked as candidate outliers in this step:") )
    print(D_filterList)
    
    ## Outliers are the intersect of three candidates list
    outliersList <- c()
    outliersList <- intersect(RLEFilterList,intersect(HCoutliers,D_filterList))
    print("Outliers:")
    outliersList
    outliersIndex <- which(colnames(logtpm) %in% outliersList)
    if(!length(outliersIndex) == 0){
    logtpm <- logtpm[,-outliersIndex]
    }
    
    ## Reverse the log10 transformation to keep it tpm
    expr <- 10^logtpm
    expr = expr%>%as_tibble(rownames = "gene_ID")
    expr%>%write_delim("$[_output]","\t")

In [2]:
[postprocessing]
input: output_from("QC"), gene_count_file
output: f'{wd}/{name}.qced.tpm.gct',f'{wd}/{name}.qced.geneCount.gct'
bash: expand = "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout',container = container
    echo "# 
          #  " > $[_output[0]:r]
    cp $[_output[0]:r] $[_output[1]:r]
R: expand = "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout',container = container
    library("tidyverse")
    tpm = read_delim($[_input[0]:r],delim = "\t")
    geneCount = read_delim($[_input[1]]:r],delim = "\t")
    ## Make geneCount consistant with tpm
    geneCount = geneCount%>%filter(gene_ID %in% tpm$gene_ID)%>%select(colnames(tpm))
    ## Save each file with 3 header line
    tpm%>%write_delim($[_output[0]:r],delim = "\t", append = T)
    geneCount%>%write_delim($[_output[1]:r],delim = "\t", append = T)